In [1]:
import sys
sys.path.append('..')

In [2]:
from pathlib import Path

from azure_blob.azure_blob_helper import AzureBlobHelper
from azure_blob.read_pdf import PDFHelper
from config_notebooks import *
from azure_ai_search.azure_ai_vector_search import CustomAzureSearch

c:\Ambarish\text2sql\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
SAVED_FOLDER = '../saved_files'
FILE_NAME = 'CHAP04-BIOLOGY-CLASS11.pdf'

In [4]:
search = CustomAzureSearch(AZURE_SEARCH_SERVICE_ENDPOINT,
                            AZURE_SEARCH_ADMIN_KEY,
                            AZURE_SEARCH_INDEX_NAME,
                            NUMBER_OF_RESULTS_TO_RETURN,
                            NUMBER_OF_NEAR_NEIGHBORS,
                            MODEL_NAME,
                            EMBEDDING_FIELD_NAME,
                            AZURE_SEARCH_SEMANTIC_CONFIG_NAME)

In [5]:
save_path = Path(SAVED_FOLDER, FILE_NAME)
azure_blob_helper = AzureBlobHelper(AZ_ST_ACC_NAME, AZ_ST_ACC_KEY, AZ_ST_CONTAINER_NAME)
file_name = FILE_NAME
full_path = os.path.join(SAVED_FOLDER, FILE_NAME)
pdf_helper = PDFHelper(full_path,
                        azure_blob_helper)
pdf_helper.write_pdf()
page_map = pdf_helper.get_document_text(full_path)
sections = pdf_helper.create_sections(file_name, page_map)
batch = []
for section in sections:
    section_embeddings = search.get_embedding_query_vector(section['content'])
    section[EMBEDDING_FIELD_NAME] = section_embeddings.tolist()
    batch.append(section)  
search.upload_documents(batch)